# Эмбеддинги на основе однокоренных слов

In [ ]:
!pip install wget
!pip install morfessor
!pip install russian_tagsets
!pip install deeppavlov
!pip install wordfreq
!pip install gensim==3.8.2 

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=a2c17fa08e8449c0983799fb55c7bd2d175a3b9bc084611e5c78bd61fbbc7c36
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
  Created wheel for russian-tagsets: filename=russian_tagsets-0.6-cp36-none-any.whl size=24636 sha256=de34253da4308697f823efcb5f7fbcbfd95e68fac1b2f84466c9bb18059773ff
  Stored in directory: /root/.cache/pip/wheels/e8/9d/dd/4679aca4031fdb0d3ad65e165ba5343e61441ed7ad587a08e6
Successfully built russian-tagsets
     |████████████████████████████████| 849kB 2.9MB/s 
     |████████████████████████████████| 1.5MB 17.3MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 512kB 20.3MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 665kB 18.6MB/s 
     |████████████████████████████████| 10.4MB 32.3MB/s 
     |██████████████████████

     |████████████████████████████████| 32.8MB 126kB/s 
     |████████████████████████████████| 4.9MB 38.9MB/s 
     |████████████████████████████████| 276kB 44.2MB/s 
  Created wheel for wordfreq: filename=wordfreq-2.3.2-cp36-none-any.whl size=32817238 sha256=ea7b97f5c0aae57476a0fd5bcfca614416392b1a2b0afcab7f45a957876afc52
  Stored in directory: /root/.cache/pip/wheels/8d/ba/84/ba6be76208bd2c2124b6586f7967fb87e9f9fb4b4827e5e2c9
  Created wheel for langcodes: filename=langcodes-2.0.0-cp36-none-any.whl size=5044047 sha256=347d9de7d316209ec49360fff4381eccfefcf297d23a0cdc83b649d32fc5940c
  Stored in directory: /root/.cache/pip/wheels/c9/11/90/c7bba8118f3674d75e1457537635266a12538cf622a4684bb2
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=861428 sha256=9788d86eddd7d1ad596ab17ded465d6dbfbdac23c1fb3700e35d85fe572b3f1b
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
Successfully built wor

In [ ]:
import os
import re 
import random
import json
import copy
import wordfreq
import numpy as np
import morfessor
import gensim
from deeppavlov import build_model, configs
from scipy.stats import spearmanr

## Морфемный анализ словаря word2vec модели

In [ ]:
def read_morphemic_analyses(filename, words, probs):
    '''Функция для считывания результатов морфемного анализа.

    Аргументы функции:
    - filename - имя файла, в котором записаны результаты морфемного анализа
    - words - список слов (с частеречными тегами), для которых проведён морфемный анализ
    - probs - True, если в файле с результатами морфемного анализа есть вероятности морфов,
    иначе - False

    Функция возвращает cловарь, ключ которого - слово (с частеречным тегом), значение - разбор.
    Разбор - список морфов. 
    Каждый из морфов в свою очередь задан в виде списка: 
    0-ой элемент - морф, 
    1-ый элемент - тип морфа,
    2-ой элемент - вероятность морфа (присутствует, если аргумент probs=True)'''

    with open(filename) as f:
        data = f.read().strip().split('\n')
    
    if probs:
        analyses = [[m.split('\t') for m in '\t'.join(w.split('\t')[1:-1]).split('/')] for w in data]
        probs = [[float(n) for m in w.split('\t')[-1].split('/') for n in m.split()] for w in data]
        analyses_with_probs = [[m+[probs[iw][im]] for im, m in enumerate(w)]
                for iw, w in enumerate(analyses)] 
        data = dict(zip(words, analyses_with_probs))
    else:
        analyses = [[m.split('\t') for m in '\t'.join(w.split('\t')[1:]).split('/')] for w in data]
        data = dict(zip(words, analyses))
    return data

In [ ]:
def morphemic_analysis(words, dataset):
    '''Функция для выполнения морфемного анализа.

    Аргументы функции:
    - words - список слов (с частеречными тегами), для которых проведён морфемный анализ
    - dataset - название датасета, для слов которого выполняется морфемный анализ

    Что делает функция:
    1) записывает список слов в файл, удаляя их частеречные теги
    2) создаёт для данного датасета файл с конфигурацией
    3) выполняет морфемный анализ слов из этого датасета c помощью моделей Morfessor и CNN
    4) считывает результаты морфемного анализа
     
    Функция возвращает cловарь, ключ которого - слово, значение - разбор.
    Разбор - список морфов. 
    Каждый из морфов в свою очередь задан в виде списка: 
    0-ой элемент - морф, 
    1-ый элемент - тип морфа,
    2-ой элемент - вероятность морфа (если probs=True)'''
    
    if not os.path.isfile('MorphemicEmbeddings/Data/{}.txt'.format(dataset)):
        print('Saving {} for morphemic analysis...'.format(dataset))
        with open('MorphemicEmbeddings/Data/{}.txt'.format(dataset), 'w') as f:
            f.write('\n'.join([w.split('_')[0] for w in words]))

    if not os.path.isfile('MorphemicEmbeddings/MorfessorMorphSegm/results/res_{}.txt'.format(dataset)):
        print('Segmenting {} into morphemes with Morfessor model...'.format(dataset))
        io = morfessor.MorfessorIO()
        model_morfessor = io.read_binary_model_file(
            'MorphemicEmbeddings/MorfessorMorphSegm/models/notlemma_types_alpha0.1_beta1000.bin')  
        
        analyses = []
        for word in words:
            word = word.split('_')[0]
            try:
                morphs = model_morfessor.segment(word)
            except KeyError:
                morphs = model_morfessor.viterbi_segment(word)[0]
            analysis = '/'.join([m+'\t'+'AFFIX'  if m in affixes else m+'\t'+'ROOT' for m in morphs])
            analyses.append(analysis)
            
        with open('MorphemicEmbeddings/MorfessorMorphSegm/results/res_{}.txt'.format(dataset),
                  'w', encoding='utf-8') as f:
            f.write('\n'.join([words[i] +'\t'+analyses[i] for i in range(len(words))]))

    data_morfessor = read_morphemic_analyses( 
        'MorphemicEmbeddings/MorfessorMorphSegm/results/res_{}.txt'.format(dataset), words, probs=False)
         
    if not os.path.isfile('MorphemicEmbeddings/NeuralMorphSegm/config/config_{}.txt'.format(dataset)):
        print('Creating configuration file for CNN model...')
        morph_config = {"load_file": "MorphemicEmbeddings/NeuralMorphSegm/models/model15.json",
                "model_file": "MorphemicEmbeddings/NeuralMorphSegm/model15.hdf5",
                "test_file": "MorphemicEmbeddings/Data/{}.txt".format(dataset),
                "outfile": "MorphemicEmbeddings/NeuralMorphSegm/results/res_{}.txt".format(dataset)}
                            
        with open("MorphemicEmbeddings/NeuralMorphSegm/config/config_{}.json".format(dataset), 'w') as f:
            json.dump(morph_config, f)

    if not os.path.isfile('MorphemicEmbeddings/NeuralMorphSegm/results/res_{}.txt'.format(dataset)):
        print('Segmenting {} into morphemes with CNN model...'.format(dataset)) 
        !python "MorphemicEmbeddings/NeuralMorphSegm/neural_morph_segm.py" "MorphemicEmbeddings/NeuralMorphSegm/config/config_{dataset}.json"

    data_cnn = read_morphemic_analyses(
        'MorphemicEmbeddings/NeuralMorphSegm/results/res_{}.txt'.format(dataset), words, probs=True)
    return data_morfessor, data_cnn

In [ ]:
def load_model(model_type, corpus):
    '''Функция для скачивания и загрузки предобученных эмбеддингов слов.

    Аргументы функции:
    - model_type - тип модели эмбеддингов ('word2vec' или 'fasttext')
    - corpus - название корпуса, на котором обучалась модель эмбеддингов 
    ('RNC', 'Araneum', 'Tayga')

    Функция создаёт папку {model_type}{corpus} и помещает туда файлы, скачанные по ссылке.
    Если скачанный файл имеет расширение zip или tgz, он распаковывается.
    Если среди полученных файлов есть файл с расширением vec, txt или vec.gz,
    считываются векторы из этого файла.
    Если такого файла нет, модель загружается из файла с расширением model.

    Функция возвращает загруженную модель.'''

    folder = '{}{}'.format(model_type, corpus)

    if not os.path.exists(folder): 
        os.mkdir(folder)
        print('Downloading {} model ({})...'.format(model_type, corpus))
        url = models[model_type][corpus]
        filename = !(basename "$url")
        filename = filename[0]
        !wget -O {filename} "$url" 

        if filename.endswith('.zip'):
            !unzip {filename} -d {folder}
            !rm {filename}
        elif filename.endswith('.tgz'):
            !tar xzf {filename} -C {folder}  
            !rm {filename}    
        else:
            !mv {filename} {folder}

    filenames = os.listdir(folder) 
    
    print('Loading {} model ({})...'.format(model_type, corpus))
    for filename in filenames:
        if filename.endswith('.vec') or filename.endswith('.txt') or filename.endswith('.vec.gz'):
            model = gensim.models.KeyedVectors.load_word2vec_format('{}/{}'.format(folder, filename))
            break
    else:
        for filename in filenames:
            if filename.endswith('.model'):
                model = gensim.models.KeyedVectors.load('{}/{}'.format(folder, filename))
    return model

In [ ]:
def load_model_preprocess_vocab(corpus):
    '''Функция, которая:
    1) загружает word2vec модель, обученную на указанном корпусе
    2) формирует список слов, для которых выполняется морфемный анализ, и записывает их в файл.
    Это слова из словаря word2vec модели, 
    которые могут содержать только буквы русского языка и дефисы
    3) выполняет морфемный анализ этих слов
    
    Аргументы функции:
    - corpus - название корпуса, на котором обучалась модель эмбеддингов 
    ('RNC', 'Araneum', 'Tayga')
    
    Функция возвращает:
    1) word2vec модель
    2) список морфемных разборов для слов. 
    Для каждого слова разбор - это список морфов. 
    Каждый из морфов в свою очередь задан в виде списка: 
    0-ой элемент - морф, 
    1-ый элемент - тип морфа,
    2-ой элемент - вероятность морфа (если для морфемного анализа используется CNN-модель)'''
    
    model = load_model('word2vec', corpus)
    reg_exp = re.compile('[А-Яа-яЁё-]+')
    words = list(model.vocab.keys())
    words = [w for w in words if re.fullmatch(reg_exp, w.split('_')[0])]
    data_w2v_morfessor, data_w2v_cnn = morphemic_analysis(words, 'word2vec_{}_vocab'.format(corpus))
    return model, data_w2v_morfessor, data_w2v_cnn

## Морфемный анализ OOV-слов

In [ ]:
model_tagging = build_model(configs.morpho_tagger.UD2_0.morpho_ru_syntagrus_pymorphy, download=True)

2020-07-15 19:42:57.934 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/datasets/UD2.0_source/ru_syntagrus.tar.gz to /root/.deeppavlov/downloads/UD2.0_source/ru_syntagrus.tar.gz
100%|██████████| 18.0M/18.0M [00:04<00:00, 4.47MB/s]
2020-07-15 19:43:02.251 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /root/.deeppavlov/downloads/UD2.0_source/ru_syntagrus.tar.gz archive into /root/.deeppavlov/downloads/UD2.0_source/ru_syntagrus
2020-07-15 19:43:03.566 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/morpho_tagger/UD2.0/ru_syntagrus.tar.gz to /root/.deeppavlov/models/morpho_tagger/UD2.0/ru_syntagrus.tar.gz
100%|██████████| 30.7M/30.7M [00:06<00:00, 4.84MB/s]
2020-07-15 19:43:10.188 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /root/.deeppavlov/models/morpho_tagger/UD2.0/ru_syntagrus.tar.gz archive into /root/.deeppavlov/

2020-07-15 19:43:14.132 INFO in 'deeppavlov.models.morpho_tagger.morpho_tagger'['morpho_tagger'] at line 166: 99 symbols, 711 tags in CharacterTagger


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2020-07-15 19:43:15.538 INFO in 'deeppavlov.models.morpho_tagger.morpho_tagger'['layer_utils'] at line 192: Model: "model"
2020-07-15 19:43:15.538 INFO in 'deeppavlov.models.morpho_tagger.morpho_tagger'['layer_utils'] at line 193: __________________________________________________________________________________________________
2020-07-15 19:43:15.540 INFO in 'deeppavlov.models.morpho_tagger.morpho_tagger'['layer_utils'] at line 190: Layer (type)                    Output Shape         Param #     Connected to                     
2020-07-15 19:43:15.545 INFO in 'deeppavlov.models.morpho_tagger.morpho_tagger'['layer_utils'] at line 195: ==================================================================================================
2020-07-15 19:43:15.547 INFO in 'deeppavlov.models.morpho_tagger.morpho_tagger'['layer_utils'] at line 190: input_1 (InputLayer)            [(None, None, 32)]   0                                            
2020-07-15 19:43:15.548 INFO in 'deeppavlov.model

In [ ]:
def add_pos_tag(w):
    '''Функция, которая добавляет к слову его частеречный тег'''
    return w + '_' + model_tagging([w])[0].split('\t')[2]

In [ ]:
def tag_word_simimilarity_data(filename):
    '''Функция, которая считывает датасет для определения семантической близости
    и добавляет для слов частеречные теги.
    
    Функция принимает на вход имя файла - датасета для определения семантической близости.
    Функция возвращает список слов из этого датасета с добавленными частеречными тегами.'''

    word_similarity_tagged, wordlist_similarity_tagged = [], []

    with open('MorphemicEmbeddings/SemanticSimilarity/{}'.format(filename)) as f:
        word_similarity = f.read().strip().split('\n')
        if filename.endswith('.tsv'):
            separator = '\t'
        elif filename.endswith('.csv'):
            separator = ','
            word_similarity = word_similarity[1:]
            
        for line in word_similarity:
            w1, w2, similarity_value = line.split(separator)
            w1, w2 = add_pos_tag(w1), add_pos_tag(w2)
            wordlist_similarity_tagged.extend([w1, w2])  
            word_similarity_tagged.append('{}'.format(separator).join([w1, w2, similarity_value]))

    if not os.path.isfile(
        'MorphemicEmbeddings/SemanticSimilarity/{}_tagged.{}'.format(*filename.split('.'))):
        with open('MorphemicEmbeddings/SemanticSimilarity/{}_tagged.{}'.format(
            *filename.split('.')), 'w') as f:
            f.write('\n'.join(word_similarity_tagged))
    
    return wordlist_similarity_tagged

In [ ]:
def oov_morphemic_analysis(model, corpus):
    '''Функция для морфемного анализа внесловарных слов 
    (слов, отсутствующих в данной word2vec модели).
    
    Аргументы функции:
    - model - модель эмбеддингов
    - corpus - название корпуса, на котором обучалась модель эмбеддингов 
    ('RNC', 'Araneum', 'Tayga')
    
    Функция возвращает результаты морфемного анализа для данных слов,
    полученные с помощью моделей Morfessor и CNN'''

    oov = [w for w in all_wordlists_similarity if w not in model.vocab]
    data_rare_words_morfessor, data_rare_words_cnn = morphemic_analysis(
        oov, 'rare_words_{}'.format(corpus))
    return data_rare_words_morfessor, data_rare_words_cnn

## Получение эмбеддингов для OOV-слов на основе однокоренных слов

In [ ]:
def root_words_dict(data, probs):
    '''Функция создаёт словарь с однокоренными словами. 
    
    Аргументы функции:
    - data - словарь "слово - его морфемный анализ"
    - probs - True, если в морфемном анализе слова присутствуют вероятности морфовЮ
    иначе - False

    Функция возвращает словарь с однокоренными словами.
    Ключ словаря - корень, значение - список из кортежей.
    Каждый кортеж содержит два элемента: 
    0-ой элемент - слово (с частеречным тегом), в котором есть этот корень ,
    1-ый элемент - вероятность того, что в этом слове такой корень'''
    
    root_words = {}
    
    if probs:
        for word, morphs in data.items():
            for m in morphs:
                morph_text, morph_type, morph_prob = m
                if morph_type == 'ROOT':
                    if morph_text not in root_words:
                        root_words[morph_text] = [(word, morph_prob)]
                    else:
                        root_words[morph_text].append((word, morph_prob))
    else:
        for word, morphs in data.items():
            for m in morphs:
                morph_text, morph_type = m
                if morph_type == 'ROOT':
                    if morph_text not in root_words:
                        root_words[morph_text] = [word]
                    else:
                        root_words[morph_text].append(word)
    return root_words

In [ ]:
def get_weighted_emb(model, cognate_words, weights):
    '''Функция для получения взвешенного эмбеддинга для одного слова
    
    Аргументы функции:
    - model - используемая word2vec модель
    - cognate_words - список однокоренных слов
    - weights - список весов для каждого из этих однокоренных слов

    Функция возвращает взвешенный эмбеддинг для одного слова'''

    emb_weighted = np.zeros((model.vector_size, len(cognate_words)))

    for i, w in enumerate(cognate_words):
        emb_weighted[:, i] = model[w] * weights[w]

    sum_weights = sum(list(weights.values()))
    emb_weighted = np.sum(emb_weighted, axis=1)/sum_weights
    return emb_weighted

In [ ]:
def morphemic_f1(data_oov, data_vocab, target_word, cognate_word):
    '''Функция, которая вычисляет "морфемную F-меру" - 
    это метрика, которая характеризует то, насколько совпадают 
    морфы исходного слова и морфы для слова, которое является для него однокоренным.

    - True positive: кол-во морфов, совпадающих в исходном слове и однокоренном  для него. 
    Корень берётся с весом 1, префикс, суффикс, постфикс – с весом 0.3, 
    окончания и соединительные гласные не учитываются 

    - False positive: кол-во морфов, которые есть в однокоренном слове, 
    но отсутствуют в исходном. Веса берутся аналогично

    - True negative: кол-во морфов, которые есть в исходном слове слове, 
    но отсутствуют в однокоренном для него. Веса берутся аналогично
    
    
    Аргументы функции: 
    - data_oov - словарь "слово - его морфемный анализ" для OOV-слов, 
    - data_vocab - словарь "слово - его морфемный анализ" для слов,
    которые есть в словаре word2vec модели
    - target_word - OOV-слово, для которого мы хотим получить эмбеддинг
    - cognate_word - слово, однокоренное для данного
    
    Функция возвращает морфемную F-меру'''

    weight = {'ROOT': 1, 'PREF': 0.3, 'SUFF': 0.3, 'POSTFIX': 0.3}
    morphs_target_word = set([m[0]+'_'+str(m[1]) for m in data_oov[target_word]
                             if m[1] in {'ROOT', 'PREF', 'SUFF', 'POSTFIX'}])
    morphs_cognate_word = set([m[0]+'_'+str(m[1]) for m in data_vocab[cognate_word]
                             if m[1] in {'ROOT', 'PREF', 'SUFF', 'POSTFIX'}])
    
    def weighted_sum(lst):
        return sum([weight[el.split('_')[1]] for el in lst])

    tp = weighted_sum(morphs_target_word & morphs_cognate_word)
    fp = weighted_sum(morphs_cognate_word - morphs_target_word)
    tn = weighted_sum(morphs_target_word - morphs_cognate_word)
    precision = tp/(tp+fp)
    recall = tp/(tp+tn)
    f1 = 2*precision*recall/(precision+recall)
    return f1

In [ ]:
def new_embeddings(model, morphemic_model, data_w2v, data_rare_words, root_words_dct):
    '''Создание новых эмбеддингов на основе однокоренных слов. 
    Функция возвращает OOV-слова, для которых были получены новые эмбеддинги, и сами эмбеддинги.
    
    Способы получения эмбеддингов:

    - Word2vec + averaged: усреднение эмбеддингов всех однокоренных слов

    - Word2vec + frequency weighted: использование всех однокоренных слов с частотными весами

    - Word2vec + probability weighted: 
    использование всех однокоренных слов с весами-вероятностями того, 
    что в данном слове был верно выделен корень 

    - Word2vec + averaged with top frequencies: 
    на каждый корень слова берётся по 5 однокоренных слов
     с наибольшей частотой, эмбеддинги выбранных слов усредняются 

    - Word2vec + averaged with top probabilities: 
    на каждый корень слова берётся по 5 однокоренных слов 
    с наибольшей вероятностью того, что в данном слове был верно выделен корень,
     эмбеддинги выбранных слов усредняются

    - Word2vec + averaged with top morphemic F1 and frequency: 
    на каждый корень слова берётся по 3 однокоренных слова 
    с наибольшим значением «морфемной F1» и наибольшей частотой, 
    эмбеддинги выбранных слов усредняются

    - Word2vec + averaged with top morphemic F1 and probability:
    на каждый корень слова берётся по 3 однокоренных слова 
    с наибольшим значением «морфемной F1» и наибольшей вероятностью того, 
    что в данном слове был верно выделен корень,
    эмбеддинги выбранных слов усредняются'''
    
    if morphemic_model == 'Morfessor':
        new_words, embs_avg, embs_freq, embs_avg_top_freqs = [], [], [], []

        for target_word, target_word_morphs in data_rare_words.items():
            # находим корень/корни в редком слове
            roots = [morph_text for (morph_text, morph_type) in target_word_morphs 
                     if morph_type == 'ROOT']
            cognate_words, cognates_top_freqs = [], []
            cognates_freqs = {}

            if roots:  # если в слове был найден хотя бы один корень
                # если для этого корня были найдены однокоренные слова, которые есть в word2vec модели
                for root in roots:
                    if root_words_dct.get(root):
                        cognates_freqs[root] = {}
                        for cognate in root_words_dct.get(root):
                            cognate_words.append(cognate)
                            cognates_freqs[root][cognate] = wordfreq.word_frequency(
                                cognate.split('_')[0], 'ru') 
                        cognates_freqs_top_freqs = {k: v for k, v in sorted(cognates_freqs[root].items(),
                            key=lambda item: item[1], reverse=True)[:5]}       
                        cognates_top_freqs.extend(list(cognates_freqs_top_freqs.keys()))
            
            if cognate_words:
                new_words.append(target_word)
                embs_avg.append(np.mean([model[w] for w in set(cognate_words)], axis=0))
                embs_avg_top_freqs.append(np.mean([model[w] for w in set(cognates_top_freqs)], axis=0))
            
                freq_weights = {w: wordfreq.word_frequency(w.split('_')[0], 'ru') for w in cognate_words}
                embs_freq.append(get_weighted_emb(model, cognate_words, freq_weights))

        return new_words, embs_avg, embs_freq, embs_avg_top_freqs
        
    elif morphemic_model == 'CNN':
        new_words, embs_avg, embs_freq, embs_prob, \
        embs_avg_top_probs, embs_avg_top_freqs, \
        embs_avg_f1_freq, embs_avg_f1_prob = [], [], [], [], [], [], [], []

        for target_word, target_word_morphs in data_rare_words.items():
            # находим корень/корни в редком слове
            roots = [morph_text for (morph_text, morph_type, morph_prob) in target_word_morphs
                    if morph_type == 'ROOT']
            cognate_words, cognates_top_freqs, cognates_top_probs, \
            probs_top_freqs, probs_top_probs = [], [], [], [], []
            cognates_f1, cognates_probs, cognates_freqs = {}, {}, {}
            cognates_f1_freqs, cognates_f1_probs = {}, {}

            if roots:  # если в слове был найден хотя бы один корень
                # если для этого корня были найдены однокоренные слова, которые есть в word2vec модели
                for root in roots:
                    if root_words_dct.get(root):
                        cognates_probs[root], cognates_freqs[root], cognates_f1[root] = {}, {}, {}
                        for (cognate, prob) in root_words_dct.get(root):
                            cognate_words.append(cognate)
                            cognates_freqs[root][cognate] = wordfreq.word_frequency(
                                cognate.split('_')[0], 'ru')
                            cognates_probs[root][cognate] = prob
                            cognates_f1[root][cognate] = morphemic_f1(
                                data_rare_words, data_w2v, target_word, cognate)
                        cognates_f1_freq = {k: v for k, v in sorted(cognates_f1[root].items(), 
                            key=lambda item: (item[1], cognates_freqs[root][item[0]]), reverse=True)[:3]}
                        cognates_f1_prob = {k: v for k, v in sorted(cognates_f1[root].items(), 
                            key=lambda item: (item[1], cognates_probs[root][item[0]]), reverse=True)[:3]}
                        
                        for (k, v) in cognates_f1_freq.items():
                            cognates_f1_freqs[k] = v
                        
                        for (k, v) in cognates_f1_prob.items():
                            cognates_f1_probs[k] = v

                        cognates_probs_top_probs = {k: v for k, v in sorted(cognates_probs[root].items(),
                            key=lambda item: item[1], reverse=True)[:5]}      
                        cognates_top_probs.extend(list(cognates_probs_top_probs.keys()))

                        cognates_freqs_top_freqs = {k: v for k, v in sorted(cognates_freqs[root].items(),
                            key=lambda item: item[1], reverse=True)[:5]}       
                        cognates_top_freqs.extend(list(cognates_freqs_top_freqs.keys()))
            
            if cognate_words:
                new_words.append(target_word)
                embs_avg.append(np.mean([model[w] for w in set(cognate_words)], axis=0))
                embs_avg_top_probs.append(np.mean([model[w] for w in set(cognates_top_probs)], axis=0))
                embs_avg_top_freqs.append(np.mean([model[w] for w in set(cognates_top_freqs)], axis=0))
                embs_avg_f1_freq.append(np.mean([model[w] for w in cognates_f1_freqs], axis=0))
                embs_avg_f1_prob.append(np.mean([model[w] for w in cognates_f1_probs], axis=0))
    
                freq_weights = {w: wordfreq.word_frequency(w.split('_')[0], 'ru') for w in cognate_words}
                embs_freq.append(get_weighted_emb(model, cognate_words, freq_weights))

                prob_weights = {k: v for el in cognates_probs.values() for (k, v) in el.items()}
                embs_prob.append(get_weighted_emb(model, cognate_words, prob_weights))
                
        return new_words, embs_avg, embs_freq, embs_prob, embs_avg_top_probs, embs_avg_top_freqs, embs_avg_f1_freq, embs_avg_f1_prob

## Применение полученных эмбеддингов для определения семантической близости

In [ ]:
def print_most_similar(model, word):
    '''Функция, которая печатает слова, которые данная модель считает ближайшими к данному слову.
    
    Аргументы функции:
    - model - модель эмбеддингов
    - word - слово, для которого подбираются ближайшие слова'''
    
    print('\nWords similar to {}:'.format(word))
    similar = model.most_similar(word)
    print('\n'.join(['\t'.join([w, str(round(similarity, 4))]) for (w, similarity) in similar]))

In [ ]:
def evaluate(model, model_name, morphemic_model, corpus, 
             similarity_datasets, words2add, embeddings2add):
    '''Функция, которая оценивает качество модели для определения семантической близости.

    Аргументы функции:
    - model - модель эмбеддингов
    - model_name - название модели эмбеддингов
    - morphemic model - модель морфемного анализа
    ('Morfessor', 'CNN', None - для бейзлайнов)
    - corpus - название корпуса, на котором обучалась модель эмбеддингов 
    ('RNC', 'Araneum', 'Tayga')
    - similarity_datasets - названия файлов-датасетов для определения семантической близости

    Что делает функция:
    1) cчитывает файл с парами слов и оценками семантической близости для них
    2) записывает в файл предсказанные значения семантической близости
    3) выводит на экран:
    - корреляцию результатов с экспертной оценкой (корреляция Спирмана)
    - p-value
    - долю OOV-пар
    4) печатает ближайшие слова для указанных слов'''

    for similarity_dataset in similarity_datasets:
        print('\n{} (corpus: {}, morphemic_model: {})'.format(model_name, corpus, morphemic_model))
        print(similarity_dataset.split('.')[0])
        true_similarities, pred_similarities, results = [], [], []
        oov_ratio = 0
            
        with open('MorphemicEmbeddings/SemanticSimilarity/' + similarity_dataset) as f:
            dataset = f.read().strip().split('\n')
        if similarity_dataset.endswith('.tsv'):
            separator = '\t'
        elif similarity_dataset.endswith('.csv'):
            separator = ','
            dataset = dataset[1:]
        
        new_model = copy.deepcopy(model)
        if model_name not in ('Word2vec baseline', 'Word2vec random_vector4unknown',
                              'Word2vec random_similarity4unknown', 'FastText'):
            for i, w in enumerate(words2add):
                if w in rare_words[similarity_dataset]:
                    new_model.add(w, embeddings2add[i])
        
        for line in dataset: 
            w1, w2, true_similarity = line.replace('ё', 'е').split(separator)
            if similarity_dataset.endswith('.tsv'):
                true_similarity = float(true_similarity)/10
            try:
                pred_similarity = new_model.similarity(w1, w2)
            except KeyError:
                if model_name == 'Word2vec random_similarity4unknown':
                    pred_similarity = random.random()
                elif model_name == 'Word2vec random_vector4unknown':
                    if w1 not in new_model.vocab:
                        new_model.add([w1], [np.random.uniform(0, 1, size=300)])
                    if w2 not in new_model.vocab:
                        new_model.add([w2], [np.random.uniform(0, 1, size=300)])
                    pred_similarity = new_model.similarity(w1, w2)
                else:
                    pred_similarity = 0
                    oov_ratio += 1  
            results.append('\t'.join([w1, w2, str(pred_similarity)]))
            true_similarities.append(true_similarity)
            pred_similarities.append(pred_similarity)
            
        with open('MorphemicEmbeddings/SemanticSimilarity/{}-{}-{}-{} results.tsv'.format(
            model_name, corpus, morphemic_model, similarity_dataset.split('.')[0]), 'w') as f:
            f.write('\n'.join(results))

        oov_ratio /= len(dataset)
        spearman_corr, pvalue = spearmanr(true_similarities, pred_similarities)
        print('Spearman correlation for human annotation: {:.4f}'.format(spearman_corr))
        print('p-value: {:.4e}'.format(pvalue))
        print('OOV-pairs ratio: {:.4f}'.format(oov_ratio))
        
        wordlist = words_for_most_similar.get(similarity_dataset)
        if model_name not in ('Word2vec baseline', 'Word2vec random_similarity4unknown') and wordlist:
            for w in wordlist:
                if model_name == 'Fasttext':
                    w = w.split('_')[0]
                print_most_similar(new_model, w)
        print('-'*100)

In [ ]:
def experiment_for_one_corpus(corpus):
    '''Функция, которая получает эмбеддинги на основе однокоренных слов и 
    оценивает их на задаче определения семантической близости.
    Модель принимает на вход название корпуса, на котором были обучены эмбеддинги.'''

    model_w2v, data_w2v_morfessor, data_w2v_cnn = load_model_preprocess_vocab(corpus)
    data_rare_words_morfessor, data_rare_words_cnn = oov_morphemic_analysis(model_w2v, corpus)
    model_fasttext = load_model('fasttext', corpus)

    print('\nEVALUATING ON SEMANTIC SIMILARITY TASK')
        
    evaluate(model_w2v, 'Word2vec baseline', 'None', corpus, 
             word_sim_datasets_tagged, None, None)
        
    new_model = copy.deepcopy(model_w2v)
    evaluate(new_model, 'Word2vec random_vector4unknown', 'None', corpus, 
             word_sim_datasets_tagged, None, None)
    new_model.save_word2vec_format(
        'MorphemicEmbeddings/Word2vec random_vector4unknown {}.vec'.format(corpus))

    evaluate(model_w2v, 'Word2vec random_similarity4unknown', 'None', 
             corpus, word_sim_datasets_tagged, None, None)
    evaluate(model_fasttext,  'FastText', 'None', 
             corpus, word_sim_datasets, None, None)
        
    # словарь, который ставит в соответствие корню слова из word2vec модели, в которых он есть
    root_words_dct = root_words_dict(data_w2v_morfessor, probs=False)
    # получаем эмбеддинги на основе однокоренных слов
    new_words, embs_avg, embs_freq, embs_avg_top_freqs = new_embeddings(
        model_w2v, 'Morfessor', data_w2v_morfessor, data_rare_words_morfessor, root_words_dct)
         
    for model_name, embeddings in {
        'Word2vec + averaged': embs_avg,
        'Word2vec + frequency weighted': embs_freq,
        'Word2vec + averaged with top frequencies': embs_avg_top_freqs}.items():
        evaluate(new_model, model_name, 'Morfessor', corpus,
                 word_sim_datasets_tagged, new_words, embeddings)
        
        
    # словарь, который ставит в соответствие корню слова из word2vec модели, в которых он есть
    root_words_dct = root_words_dict(data_w2v_cnn, probs=True)

    new_words, embs_avg, embs_freq, embs_prob, embs_avg_top_probs,\
        embs_avg_top_freqs, embs_avg_f1_freq, embs_avg_f1_prob = new_embeddings(
                model_w2v, 'CNN', data_w2v_cnn, data_rare_words_cnn, root_words_dct)
        
    for model_name, embeddings in {
        'Word2vec + averaged': embs_avg,
        'Word2vec + frequency weighted': embs_freq,
        'Word2vec + probability weighted': embs_prob,
        'Word2vec + averaged with top frequencies': embs_avg_top_freqs,
        'Word2vec + averaged with top probabilities': embs_avg_top_probs,
        'Word2vec + averaged with top morphemic F1 and frequency': embs_avg_f1_freq,
        'Word2vec + averaged with top morphemic F1 and probability': embs_avg_f1_prob}.items():
        # получаем эмбеддинги на основе однокоренных слов
        
        evaluate(new_model, model_name, 'CNN', corpus,
                 word_sim_datasets_tagged, new_words, embeddings)
                                
    del model_w2v, model_fasttext, new_model, data_w2v_cnn, data_w2v_morfessor, data_rare_words_cnn, data_rare_words_morfessor

In [ ]:
models = {'word2vec':
                {'Tayga': 'http://vectors.nlpl.eu/repository/20/185.zip',
                'RNC': 'http://vectors.nlpl.eu/repository/20/180.zip',
                'Araneum': 'https://rusvectores.org/static/models/rusvectores4/araneum/araneum_upos_skipgram_300_2_2018.vec.gz'},
          'fasttext':
                {'Tayga': 'http://vectors.nlpl.eu/repository/20/187.zip',
                 'RNC': 'http://vectors.nlpl.eu/repository/20/181.zip',
                 'Araneum': 'https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz'}}

In [ ]:
affixes = []

for filename in {'endings.txt', 'interfixes.txt', 'postfixes.txt', 'prefixes.txt',
                 'suffixes_derivational.txt', 'suffixes_inflectional.txt'}:
    with open('MorphemicEmbeddings/MorphLists/'+filename, encoding='windows-1251') as f:
        data = f.read().split('\n')
        for affix in data:
            if '(' in affix:
                affix = affix.split('(')[0]
            affixes.append(affix)
affixes.append('-')
affixes = set(affixes)

In [ ]:
word_sim_datasets = ['rare_multimorphemic.tsv', 'ae-test.csv', 
        'ae2-test.csv', 'hj-test.csv', 'rt-test.csv']
word_sim_datasets_tagged = ['{}_tagged.{}'.format(*dataset_name.split('.'))
        for dataset_name in word_sim_datasets]
        
words_for_most_similar = {'rare_multimorphemic_tagged.tsv':
        ['восемьсотпятидесятилетие_NOUN', 'древневерхненемецкий_ADJ',
        'камнесамоцветный_ADJ', 'раннеперестроечный_ADJ',
        'шумозаградительный_ADJ', 'биокибернетик_NOUN']}

In [ ]:
wordlist_rare_multimorphemic = tag_word_simimilarity_data('rare_multimorphemic.tsv')
wordlist_aetest = tag_word_simimilarity_data('ae-test.csv')
wordlist_ae2test = tag_word_simimilarity_data('ae2-test.csv')
wordlist_hjtest = tag_word_simimilarity_data('hj-test.csv')
wordlist_rttest = tag_word_simimilarity_data('rt-test.csv')

rare_words = {'rare_multimorphemic_tagged.tsv': wordlist_rare_multimorphemic,
              'ae-test_tagged.csv': wordlist_aetest, 'ae2-test_tagged.csv': wordlist_ae2test,
              'hj-test_tagged.csv': wordlist_hjtest, 'rt-test_tagged.csv': wordlist_rttest}
              
all_wordlists_similarity = set(wordlist_rare_multimorphemic + wordlist_aetest +
        wordlist_ae2test + wordlist_hjtest + wordlist_rttest)

In [ ]:
for corpus in 'RNC', 'Araneum', 'Tayga':
    experiment_for_one_corpus(corpus)

--2020-07-15 19:46:49--  http://vectors.nlpl.eu/repository/20/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484452317 (462M) [application/zip]
Saving to: ‘180.zip’

180.zip             100%[===================>] 462.01M  22.9MB/s    in 25s     

2020-07-15 19:47:15 (18.2 MB/s) - ‘180.zip’ saved [484452317/484452317]

Archive:  180.zip
  inflating: word2vecRNC/meta.json   
  inflating: word2vecRNC/model.bin   
  inflating: word2vecRNC/model.txt   
  inflating: word2vecRNC/README      
Loading word2vec model (RNC)...
Saving word2vec_RNC_vocab for morphemic analysis...
Segmenting word2vec_RNC_vocab into morphemes with Morfessor model...
Creating configuration file for CNN model...
Segmenting word2vec_RNC_vocab into morphemes with CNN model...
Using TensorFlow backend.
2020-07-15 19:48:39.470186: I tensorflow/stream_executor/platf

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide



Word2vec + averaged (corpus: RNC, morphemic_model: Morfessor)
rare_multimorphemic_tagged
Spearman correlation for human annotation: 0.6380
p-value: 3.2389e-13
OOV-pairs ratio: 0.0000

Words similar to восемьсотпятидесятилетие_NOUN:
пролететь_VERB	0.5966
приземлиться_VERB	0.5315
взмыть_VERB	0.5245
летевать_VERB	0.5181
пролетать_VERB	0.5112
перелететь_VERB	0.5073
взмывать_VERB	0.4993
подлетать_VERB	0.4971
зависнуть_NOUN	0.4959
полураспавшийся_VERB	0.4955

Words similar to древневерхненемецкий_ADJ:
вверх_ADV	0.5767
вершина_NOUN	0.5449
центрально-черноземный_ADJ	0.5213
полуразложившийся_VERB	0.5112
вверху_ADV	0.5087
отлогой_NOUN	0.5069
сверху_ADV	0.5062
гора_NOUN	0.5036
верх_NOUN	0.5027
кверху_ADV	0.5016

Words similar to камнесамоцветный_ADJ:
самозатачивающийся_ADJ	0.8902
самоутверждение_NOUN	0.5473
стремление_NOUN	0.5288
бунтарство_NOUN	0.5134
способный_ADJ	0.4892
индивидуальность_NOUN	0.4816
своеволие_NOUN	0.4778
самосознание_NOUN	0.4743
самобытность_NOUN	0.4741
индивидуализм_NOUN	0.46

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide



Word2vec + averaged (corpus: Araneum, morphemic_model: Morfessor)
rare_multimorphemic_tagged
Spearman correlation for human annotation: 0.6818
p-value: 1.6197e-15
OOV-pairs ratio: 0.0000

Words similar to восемьсотпятидесятилетие_NOUN:
полураспавшийся_VERB	0.9183
полуразложившийся_VERB	0.9161
полузамерзший_VERB	0.9158
раннеперестроечный_ADJ	0.8896
валяносапожник_NOUN	0.8651
чехословацкий_ADJ	0.8562
камнесамоцветный_ADJ	0.8315
неизбирабельный_ADJ	0.8188
вытянутый_VERB	0.7987
древневерхненемецкий_ADJ	0.7957

Words similar to древневерхненемецкий_ADJ:
раннеперестроечный_ADJ	0.8694
полуразложившийся_VERB	0.8611
полузамерзший_VERB	0.8538
полураспавшийся_VERB	0.8507
валяносапожник_NOUN	0.8288
неизбирабельный_ADJ	0.8275
камнесамоцветный_ADJ	0.8094
чехословацкий_ADJ	0.8066
бессквозняковый_ADJ	0.8059
шумозаградительный_ADJ	0.7974

Words similar to камнесамоцветный_ADJ:
раннеперестроечный_ADJ	0.8968
полуразложившийся_VERB	0.8869
полузамерзший_VERB	0.8766
полураспавшийся_VERB	0.8756
необуржуазны

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide



Word2vec + averaged (corpus: Tayga, morphemic_model: Morfessor)
rare_multimorphemic_tagged
Spearman correlation for human annotation: 0.6419
p-value: 2.0873e-13
OOV-pairs ratio: 0.0000

Words similar to восемьсотпятидесятилетие_NOUN:
полураспавшийся_VERB	0.8769
полуразложившийся_VERB	0.8722
полузамерзший_VERB	0.8712
раннеперестроечный_ADJ	0.8335
пенополиуританин_NOUN	0.8237
неремонтопригодный_ADJ	0.8165
центрально-черноземный_ADJ	0.8138
валяносапожник_NOUN	0.8038
беллинсгаузен_NOUN	0.8023
авиационно-космический_ADJ	0.7769

Words similar to древневерхненемецкий_ADJ:
центрально-черноземный_ADJ	0.8522
полуразложившийся_VERB	0.8462
раннеперестроечный_ADJ	0.8433
полураспавшийся_VERB	0.8328
полузамерзший_VERB	0.8326
беллинсгаузен_NOUN	0.809
пенополиуританин_NOUN	0.7973
камнесамоцветный_ADJ	0.7956
морально-нравственный_ADJ	0.7786
неизбирабельный_ADJ	0.7744

Words similar to камнесамоцветный_ADJ:
самозатачивающийся_ADJ	0.9826
раннеперестроечный_ADJ	0.8801
древесностружечный_ADJ	0.8692
пенопол